In [7]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

import pandas as pd
import re

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from time import sleep
import chromedriver_autoinstaller
from selenium.webdriver.chrome.options import Options

chromedriver_autoinstaller.install()
options = Options()

#options.add_argument('headless')     # 크롬브라우저 숨기기
options.add_argument('window-size=1920,1080')  # 모니터 해상도 지정
options.add_argument('disable-gpu')  # 불필요한 GPU 기능을 제거해서 셀레니움 작동 속도를 올려주는 옵션
options.add_argument('--no-sandbox')
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=options)

def crawl_review(product_num, page_num):
    if page_num ==0:
        raise Exception("Page_num 는 1 부터 시작입니다")
        
    headers = {
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.9,ko;q=0.8',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive',
        'DNT': '1',
        'Pragma': 'no-cache',
        'Referer': 'http://www.yes24.com/Product/Goods/114853233',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
    }

    params = {
        'Sort': '1',
        'PageNumber': page_num, # 1부터 시작
        'Type': 'ALL',
        '_': '1683991433079',
    }

    response = requests.get(
        f'http://www.yes24.com/Product/CommunityModules/GoodsReviewList/{product_num}',
        params=params,
        headers=headers,
        verify=False,
    )
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

def fetch_data(cate_url):
    # Navigate to url
    driver.get(cate_url+'&FetchSize=100') # FetchSize로 top100까지 출력되게 함
    sleep(0.2)

    # Fetch data
    title_elements = driver.find_elements(By.XPATH, '//*[@id="category_layout"]/tbody/tr/td[3]/p[1]/a[1]')
    author_elements = driver.find_elements(By.XPATH, '//*[@id="category_layout"]/tbody/tr/td[3]/div/a[1]')
    price_elements = driver.find_elements(By.XPATH, '//*[@id="category_layout"]/tbody/tr/td[3]/p[2]')
    
    # Prepare data for DataFrame
    data = {
        'title': [elem.text for elem in title_elements],
        'url': [elem.get_attribute('href') for elem in title_elements],
        'author_list': [elem.text for elem in author_elements],
        'price_list': [elem.text for elem in price_elements],
    }

    # Create DataFrame
    df = pd.DataFrame(data)
    df = df.rename_axis('rank').reset_index()
    df['rank'] = df['rank'] + 1 
    return df

# Initialize an empty DataFrame
cate_url = 'http://www.yes24.com/24/category/bestseller?CategoryNumber=001&sumgb=06'
df = fetch_data(cate_url)

review_count_list = []
content_text_list = [] 
review_lists = [] # 리뷰들의 총 합

for product_url in tqdm(df['url']):
    # 리뷰 카운트 가져오기 + 책 내용 가져오기 
    response = requests.get(product_url)
    soup = BeautifulSoup(response.content, 'html.parser')    
    # 리뷰 가져오기
    try:
        review_count = soup.select('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_ratingArea > span.gd_reviewCount > a > em')[0].text
    except: 
        review_count = 0
    
    # 본문 가져오기 
    try:
        content_text = soup.find(class_ = 'txtContentText').text.strip()
    except: 
        content_text = ''
        
        
    #### 리뷰 가지고 오는 코드 ### 
    
    # 리뷰가 6개면 2페이지를 가져와야 함 
    # 올림 연산 후에 + 1
    page_num = int(review_count)//5 +1 
    
    # product_numbers를 넣어 주는 형태로 리뷰 코드 구성됨
    product_number = product_url.split('/')[-1] # 상품 코드 찾는 코드
    
    review_list = [] # 상품 마다 초기화 해서 넣어줘야함
    for i in range(1, page_num+1): # 페이지 넘버는 1부터 무조건 
        
        try:
            soup = crawl_review(product_number, i)
            review_tmp = soup.find_all(class_ = 'review_cont')
            review_tmp = review_tmp[1::2] # 리뷰 풀네임은 두번째 부터 계속 2번째에 있음
            review_tmp = [i.text.strip() for i in review_tmp]
            review_list = review_list + review_tmp
        except:
            review_list = ['']

        if i == 20: # 리뷰 100개 이상 멈추기 
            break
            
    review_count_list.append(review_count) # 요소 추가
    content_text_list.append(content_text) # 요소 추가
    review_lists.append(review_list)
    time.sleep(2)
    

df['review_count'] = review_count_list
df['contents'] = content_text_list
df['reviews'] = review_lists

df.to_excel('yes_24.xlsx', index=False)
df.to_csv('yes_24.csv', index=False)